In [ ]:
import requests
import json
import numpy as np

In [ ]:
import json
import requests
from collections import defaultdict

In [ ]:
from lxml import html
import pandas as pd

In [ ]:
df =pd.read_csv("altmetricIDS.csv", header=None)

In [ ]:
df.columns = ['Altmetrics_Id']
df.head()

In [ ]:
df["youtube_links"] = ""
df.head()

In [ ]:
ids = np.array(df['Altmetrics_Id'])
ids[5255]

In [ ]:
def add_video(alt_id):
    try:
        response = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))
        if 'video' in dict(json.loads(response.content)):
            return dict(json.loads(response.content))['video']
    except:
        return ''

In [ ]:
param = {"key" : ""}

In [ ]:
url_content = "https://www.googleapis.com/youtube/v3/videos?part=snippet&key=&id="

In [ ]:
url = "https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics"

In [ ]:
from bs4 import BeautifulSoup
for alt_id in ids[5555:5655]:
    mylist = []
    alt_req = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))
    alt_rs= json.loads(alt_req.text)
    mylist.append({str(alt_id) : alt_rs})
    y_links = []
    html = requests.get('https://www.altmetric.com/details/' + str(alt_id) + '/video').text
    bs = BeautifulSoup(html)
    possible_links = bs.find_all('a')
    for link in possible_links:
        if link.has_attr('href') and 'youtube' in link.attrs['href']:
            y_links.append(link.attrs['href'])
    youtube_data = []
    for link in y_links:
        param["id"] = link[32:]
        response_statistics = requests.get(url, param)
        print(response_statistics.status_code)
        json_statistics = json.loads(response_statistics.text)
        youtube_data.append(json_statistics)
    mylist.append({'youtube':youtube_data})
    file = open(str(alt_id) + ".txt", "w+")
    json.dump(mylist, file)
    file.close()

In [ ]:
with open("414470.txt") as json_file:
    data = json.load(json_file)
data[0]['414470'].keys()

In [ ]:
df_ = pd.DataFrame()

In [ ]:
df_.loc[0, 'altmetric_id'] = str(414470)

In [ ]:
col = ['cited_by_fbwalls_count', 'cited_by_feeds_count', 'cited_by_gplus_count', 'cited_by_msm_count', 
       'cited_by_posts_count', 'cited_by_rdts_count', 'cited_by_tweeters_count', 'cited_by_videos_count']

In [ ]:
data[0]['414470']["cited_by_posts_count"]

In [ ]:
data[1]['youtube'][1]["items"][0]["id"]

In [ ]:
for c in col:
    i=0
    df_.loc[i, c] = data[0]['414470'][c]

In [ ]:
df_.head()

In [ ]:
stats = ['viewCount', 'likeCount',  'dislikeCount', 'favoriteCount','commentCount']

In [ ]:
for stat in stats:
    for i in range(len(data[1]['youtube'])):
        try:
            df_.loc[i, stat] = data[1]['youtube'][i]['items'][0]['statistics'][stat]
        except IndexError:
            df_.loc[i, stat] = 0

In [ ]:
df_.head()

In [ ]:
for i in range(1,4):
    for c in ["altmetric_id", "cited_by_fbwalls_count","cited_by_feeds_count","cited_by_gplus_count","cited_by_msm_count","cited_by_posts_count","cited_by_rdts_count","cited_by_tweeters_count", "cited_by_videos_count"]:
        df_.loc[i, c] = df_.loc[0, c]

In [ ]:
df_.head()

In [ ]:
for i in range(4):
    df_.loc[i, "video_id"] = data[1]['youtube'][i]["items"][0]["id"]

In [ ]:
df_.head()

In [ ]:
df_.to_csv("test_.csv") 

In [ ]:
y_link = url_statistics +"GSqdhz--yR8"
rs_st = requests.get(y_link)
rs_st_d = json.loads(rs_st.text)
rs_st_d

In [ ]:
y_content = url_content + "GSqdhz--yR8"
rs_ct = requests.get(y_link)
rs_ct_d = json.loads(rs_st.text)
rs_ct_d

In [ ]:
social_media_columns = ['cited_by_fbwalls_count', 'cited_by_gplus_count', 'cited_by_msm_count', 'cited_by_posts_count', 
                        'cited_by_tweeters_count', 'cited_by_videos_count', 'cited_by_wikipedia_count', 
                        'cited_by_accounts_count',]

In [ ]:
for i in range(len(ids[:100])):
    rs = requests.get("https://api.altmetric.com/v1/id/" + str(ids[i]))
    if (rs.status_code == 200):
        rs_json = json.loads(rs.text)
        for col in social_media_columns:
            try:
                df.loc[i, col] = rs_json[col]
            except KeyError:
                df.loc[i, col] = 0

In [ ]:
df[60:80]

In [ ]:
cols = ['title', 'doi']

In [ ]:
for i in range(len(ids[:100])):
    rs = requests.get("https://api.altmetric.com/v1/id/" + str(ids[i]))
    if (rs.status_code == 200):
        rs_json = json.loads(rs.text)
        for col in cols:
            try:
                df.loc[i, col] = rs_json[col]
            except KeyError:
                df.loc[i, col] = 0

In [ ]:
df.head()

In [ ]:
df[:100].to_csv(path_or_buf = ".\\sample_dataset.csv", index=False)

In [ ]:
url_comments = "https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=_qUhIZcxNvE&key="

In [ ]:
response = requests.get(url)
print(response.status_code)

In [ ]:
json_data = json.loads(response.text)
json_data.keys()

In [ ]:
json_data["items"][2]['snippet']['topLevelComment']['snippet']

In [ ]:
lc = 0
dc = 0
for j in len(json_data['items']):
    lc + = json_data['items'][j]["likeCount"]
    dc + = json_data['items'][j]["likeCount"]

In [ ]:
# add author information
def add_all_info(alt_id):
    # create the dict
    result = defaultdict(dict)
    try:
        # make the request
        response = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))

        # add the values to the dict
        if 'authors' in dict(json.loads(response.content)):
            result['author_count'] = len(dict(json.loads(response.content))['authors'])
            result['authors'] = ', '.join(dict(json.loads(response.content))['authors'])
        else:
            result['author_count'] = ''
            result['authors'] = ''

         # add the values to the dict
        if 'doi' in dict(json.loads(response.content)):
            result['doi'] = dict(json.loads(response.content))['doi']
        else:
            result['doi'] = ''

        # add the abstract
        if 'abstract' in dict(json.loads(response.content)):
            result['abstract'] = dict(json.loads(response.content))['abstract']
        else:
            result['abstract'] = ''

        # add the pubdate
        if 'published_on' in dict(json.loads(response.content)):
            result['pub_date'] = dict(json.loads(response.content))['published_on']
        else:
            result['pub_date'] = ''

        # add the twitter posts count as of 2018/19
        if 'cited_by_tweeters_count' in dict(json.loads(response.content)):
            result['twitter_count_y18'] = dict(json.loads(response.content))['cited_by_tweeters_count']
        else:
            result['twitter_count_y18'] = ''

        # add the title of the article
        if 'title' in dict(json.loads(response.content)):
            result['title'] =  dict(json.loads(response.content))['title']
        else:
            result['title'] = ''

        # return the result
        return result
    except:
        return result

In [ ]:
r = add_all_info(702709 )

In [ ]:
r.keys()

In [ ]:
rs_ = requests.get("https://api.altmetric.com/v1/id/" + str(2394296))

In [ ]:
rs_js = json.loads(rs_.text)

In [ ]:
rs_js

In [ ]:
fl = open("temp.txt", 'w+')

In [ ]:
json.dump(rs_js, fl)

In [ ]:
rs_js['cited_by_accounts_count']

In [ ]:
rs_js['cited_by_videos_count']

In [ ]:
response_statistics = requests.get(url_statistics)
print(response.status_code)

In [ ]:
json_statistics = json.loads(response_statistics.text)
json_statistics.keys()

In [ ]:
json_statistics['items']